In [1]:
import tensorflow as tf
import numpy as np
from keras import layers
from keras import models
from keras import regularizers
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
train_dir = 'Dataset/train'
test_dir = 'Dataset/test'

max_count=100
reg_val=[]
lr_val=[]
test_loss=[]
test_acc=[]


In [3]:
# Sampling learning rate and regularization from a uniform distribution
reg=10**(np.random.uniform(-4,0))
lr=10**(np.random.uniform(-3,-4))


In [4]:
model=models.Sequential()

model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(60,60,3)))

model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3),activation='relu'))

model.add(layers.MaxPooling2D(2,2))

model.add(layers.Conv2D(128,(3,3),activation='relu'))

model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3),activation='relu'))

model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l2(reg)))

model.add(layers.Dense(7,activation='sigmoid',kernel_regularizer=regularizers.l2(reg)))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 58, 58, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 128)        0

In [6]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),metrics=['acc'])

In [7]:
#Rescale all the images by 1/255
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
            train_dir,
            target_size=(60,60),
            batch_size=20,
            class_mode='categorical')
test_generator=test_datagen.flow_from_directory(
            test_dir,
            target_size=(60,60),
            batch_size=20,
            class_mode='categorical')

Found 4011 images belonging to 7 classes.
Found 1007 images belonging to 7 classes.


In [8]:
#Fit the model using batch generator
history=model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=5,
        validation_data=test_generator,
        validation_steps=50)

reg_val.append(reg)
lr_val.append(lr)
test_loss.append(history.history['val_loss'])
test_acc.append(history.history['val_acc'])

C:\Users\rajak\AppData\Local\Temp/ipykernel_13468/1138232458.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(


Epoch 1/5
100/100 [==============================] - 102s 1s/step - loss: 0.9600 - acc: 0.6765 - val_loss: 0.0820 - val_acc: 0.9890
Epoch 2/5
100/100 [==============================] - 67s 667ms/step - loss: 0.1406 - acc: 0.9705 - val_loss: 0.0241 - val_acc: 0.9980
Epoch 3/5
100/100 [==============================] - 58s 579ms/step - loss: 0.0504 - acc: 0.9910 - val_loss: 0.0134 - val_acc: 0.9990
Epoch 4/5
100/100 [==============================] - 52s 517ms/step - loss: 0.1478 - acc: 0.9845 - val_loss: 0.0139 - val_acc: 0.9990
Epoch 5/5
100/100 [==============================] - 49s 491ms/step - loss: 0.0911 - acc: 0.9880 - val_loss: 0.0116 - val_acc: 0.9990


In [12]:
acc=history.history['acc']
test_acc=history.history['val_acc']
loss=history.history['loss']
test_loss=history.history['val_loss']
# epochs=range(1,len(acc)+1)
# plt.plot(epochs,acc,'bo',label='TRAINING ACCURACY')
# plt.plot(epochs,test_acc,'b',label='TEST ACCURACY')
# plt.title('TRAINING AND TEST ACCURACY')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.figure()
# plt.plot(epochs,loss,'bo',label='TRAINING LOSS')
# plt.plot(epochs,test_loss,'b',label='TEST LOSS')
# plt.title('TRAINING AND TESTING LOSS')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [13]:
print ("*"*30)
print ("Finding the highest Test Accuracy and lowest Test Loss...")

index1=0
index2=0
max_test_acc=max(test_acc)
min_test_loss=min(test_loss)
for i in range(max_count):
    temp1=max(test_acc)
    if(temp1>=max_test_acc):
        max_test_acc=temp1
        index1=i
    temp2=min(test_loss)
    if(temp2<min_test_loss):
        min_test_loss=temp2
        index2=i  

print ('Maximum Testing Accuracy:',max_test_acc)
print ('Minimum Testing Loss:',min_test_loss)
print ('Value of optimum learning rate :',lr_val)
print ('Value of optimum regularization:',reg_val)


******************************
Finding the highest Test Accuracy and lowest Test Loss...
Maximum Testing Accuracy: 0.9990000128746033
Minimum Testing Loss: 0.011573069728910923
Value of optimum learning rate : [0.0005211234057531751]
Value of optimum regularization: [0.00011425445656551216]


In [14]:
import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_model.h5')